In [29]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [30]:
data = pd.read_csv('winequality-red.csv')

In [31]:
data['quality'] = data['quality'].astype(np.float32)
x = data['quality']

In [32]:
quality_target = np.array(x)
quality_target

array([5., 5., 5., ..., 6., 5., 6.], dtype=float32)

In [33]:
del data['quality']

In [34]:
inputs = data.to_numpy()
inputs

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]])

In [35]:
tens_inputs = torch.from_numpy(inputs)
tens_target = torch.from_numpy(quality_target)

In [36]:
train_ds = TensorDataset(tens_inputs, tens_target)
train_ds[0:3]

(tensor([[7.4000e+00, 7.0000e-01, 0.0000e+00, 1.9000e+00, 7.6000e-02, 1.1000e+01,
          3.4000e+01, 9.9780e-01, 3.5100e+00, 5.6000e-01, 9.4000e+00],
         [7.8000e+00, 8.8000e-01, 0.0000e+00, 2.6000e+00, 9.8000e-02, 2.5000e+01,
          6.7000e+01, 9.9680e-01, 3.2000e+00, 6.8000e-01, 9.8000e+00],
         [7.8000e+00, 7.6000e-01, 4.0000e-02, 2.3000e+00, 9.2000e-02, 1.5000e+01,
          5.4000e+01, 9.9700e-01, 3.2600e+00, 6.5000e-01, 9.8000e+00]],
        dtype=torch.float64),
 tensor([5., 5., 5.]))

In [37]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

Model:

In [38]:
model = nn.Linear(11,1599)
w , b = model.parameters()
print(w)
print(b)

Parameter containing:
tensor([[-0.2792, -0.0754, -0.0228,  ..., -0.2141,  0.0333,  0.0767],
        [-0.2222, -0.0585,  0.2606,  ...,  0.1505, -0.2442, -0.1227],
        [ 0.1504,  0.0412,  0.1379,  ...,  0.1883, -0.1683, -0.2968],
        ...,
        [ 0.0073, -0.2737,  0.1991,  ...,  0.1204,  0.0093, -0.2248],
        [-0.0370, -0.1950, -0.1916,  ...,  0.2599,  0.1342, -0.0125],
        [-0.2486,  0.0466,  0.2296,  ..., -0.2768,  0.1837, -0.0230]],
       requires_grad=True)
Parameter containing:
tensor([ 0.2941, -0.0050, -0.1534,  ...,  0.0674,  0.2458, -0.0918],
       requires_grad=True)


In [43]:
loss_fn = F.mse_loss
loss = loss_fn(model(tens_inputs.float()), tens_target)
loss

C:\Users\Sarthak\AppData\Local\Temp\ipykernel_10816\1929563904.py:2: UserWarning: Using a target size (torch.Size([1599])) that is different to the input size (torch.Size([1599, 1599])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(model(tens_inputs.float()), tens_target)


tensor(142.7078, grad_fn=<MseLossBackward0>)

In [40]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [41]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [44]:
fit(500, model, loss_fn, opt,train_dl)

RuntimeError: expected scalar type Float but found Double

In [ ]:
preds = model(inputs)
preds

In [ ]:
targets

In [ ]:
# torch.save(model.state_dict(),"weights_file_name_here.pth")